Creation of Activity Schema
=======================

We need to create an activity schema with 500 to 1,000 rows meeting the following requirements:
1. It must have the design of an active schema (all activities in one table)
2. Only a strict sequence `click`&rarr; `lead` &rarr; `consultation` &rarr; `sale` should be considered, i.e., when there is a `click` &rarr; `consultation` or a `sale` &rarr; `sale` &rarr; `consultation` &rarr; `sale` situation, they must be ignored.
3. There should be several landing pages,and we are going to count conversion for them
4. Finally, we create a query to calculate conversion.

First, we import packages

In [1]:
import pandas as pd
import numpy as np
import faker as fake

We already have a table, `activity_table_peargrape.csv`, and we need to augment it.
The structure of the table is represented below:  

**Column**|**Description**|**Type**|**Required**
-----|-----|-----|-----
activity\_id|Unique identifier for the activity record|string|yes
ts|Timestamp for when the activity occurred|timestamp|yes
customer|Email as a unique identifier for the customer|string|yes
activity|Name of the activity based on funnel (`click`, `lead`, `consultation`, `sale`)|string|yes
anonymous_customer\_id|A unique customer id for cases when `customer` value is null|string|no
feature\_json|Activity-specific features|JSON|yes
revenue\_impact|Revenue or cost associated with the activity. Not null only when activity value is `sale`|float|no
link|URL of a link. . Not null only when activity value is `click` |string|no  

In [3]:
data = pd.read_csv('C:/Mike/Analytics/activity_schema_sample/activity_table_peargrape.csv')

In [4]:
data

,activity_id,ts,customer,activity,anonymous_customer_id,feature_json,revenue_impact,link
0,df3v34dd3fn49sfe,2023-06-01 09:00:00,johndoe@example.com,click,NaN,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,g48s6a29sefhcn3d,2023-06-01 09:10:00,janedoe@example.com,click,NaN,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,1d8s9a3j29snsf28,2023-06-01 09:15:00,janedoe@acme.com,click,NaN,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,f8d73dfg5nswd9f4,2023-06-01 09:20:00,alexsmith@example.com,click,NaN,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,4f3g29sf9sdnfld3,2023-06-01 09:25:00,NaN,click,segment_23kdfjdnfjn8r,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...,...,...
125,3d9gfnj4sd8f,2023-06-01 19:30:00,johndoe@example.com,click,NaN,"{""page"": ""contact_us"", ""source"": ""referral""}",NaN,https://example.com/contact-us
126,fgn3sd94j3n,2023-06-01 19:35:00,NaN,consultation,segment_abd5d3,"{""preferred_time"": ""afternoon""}",NaN,NaN
127,4nsd8g3j9n4,2023-06-01 19:40:00,janedoe@example.com,consultation,NaN,"{""preferred_time"": ""morning""}",NaN,NaN
128,jf8d4n3g9sd,2023-06-01 19:45:00,janedoe@acme.com,click,NaN,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/testimonials


In [5]:
data.describe()

,activity_id,ts,customer,activity,anonymous_customer_id,feature_json,revenue_impact,link
count,130,130,99,130,31,130,10,67
unique,130,130,6,4,10,18,4,7
top,df3v34dd3fn49sfe,2023-06-01 09:00:00,janedoe@example.com,click,segment_abd5d3,"{""form"": ""newsletter_signup""}","19,99",https://example.com/products
freq,1,1,37,67,9,18,5,18


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   activity_id            130 non-null    object
 1   ts                     130 non-null    object
 2   customer               99 non-null     object
 3   activity               130 non-null    object
 4   anonymous_customer_id  31 non-null     object
 5   feature_json           130 non-null    object
 6   revenue_impact         10 non-null     object
 7   link                   67 non-null     object
dtypes: object(8)
memory usage: 8.2+ KB


We are going to simplify it by removing columns `activity_id` and `anonymous_customer_id`. All our clients will have ids by email:

In [9]:
data = data.drop(columns=['activity_id', 'anonymous_customer_id'])

In [10]:
data

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,johndoe@example.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,janedoe@example.com,click,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,janedoe@acme.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,alexsmith@example.com,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,NaN,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...
125,2023-06-01 19:30:00,johndoe@example.com,click,"{""page"": ""contact_us"", ""source"": ""referral""}",NaN,https://example.com/contact-us
126,2023-06-01 19:35:00,NaN,consultation,"{""preferred_time"": ""afternoon""}",NaN,NaN
127,2023-06-01 19:40:00,janedoe@example.com,consultation,"{""preferred_time"": ""morning""}",NaN,NaN
128,2023-06-01 19:45:00,janedoe@acme.com,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/testimonials


Now, we are going to add 2,000 more rows with random timestamp

But first, we are going to change type of certain columns

In [15]:
data['revenue_impact'] = data['revenue_impact'].str.replace(',','.')

In [16]:
convert_dict = {
    'customer': str,
    'activity': str,
    'revenue_impact': float,
    'link': str
}

data = data.astype(convert_dict)

In [21]:
data.head(50)

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,johndoe@example.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,janedoe@example.com,click,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,janedoe@acme.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,alexsmith@example.com,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
5,2023-06-01 09:30:00,sarahjones@example.com,click,"{""page"": ""about_us"", ""source"": ""organic""}",NaN,https://example.com/about-us
6,2023-06-01 09:35:00,janedoe@example.com,click,"{""page"": ""contact_us"", ""source"": ""referral""}",NaN,https://example.com/contact-us
7,2023-06-01 09:40:00,,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
8,2023-06-01 09:45:00,johndoe@example.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
9,2023-06-01 09:50:00,sarahsmith@example.com,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products


In [20]:
data['customer'] = data['customer'].str.replace('nan', '')

In [23]:
from datetime import datetime, timedelta
import random

In [127]:
data_1 = data

In [128]:
start_date = datetime(2023, 6, 1, 0, 0, 0)
end_date = datetime.now()

# Generate 2000 dummy datetimes
dummy_datetimes = []
for _ in range(2000):
    # Generate a random datetime between the start and end dates
    random_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))

    # Append the datetime to the list
    dummy_datetimes.append(random_date)

# Convert the list of datetimes to pandas Timestamps
dummy_datetimes = pd.to_datetime(dummy_datetimes)

# Create a new DataFrame with the dummy datetimes
new_rows = pd.DataFrame({'ts': dummy_datetimes})

# Append the new rows to the existing DataFrame
data_1 = data_1.append(new_rows, ignore_index=True)

In [129]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,johndoe@example.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,janedoe@example.com,click,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,janedoe@acme.com,click,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,alexsmith@example.com,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,,click,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,NaN,NaN,NaN,NaN,NaN
2126,2023-06-28 10:26:58,NaN,NaN,NaN,NaN,NaN
2127,2023-06-19 22:47:34,NaN,NaN,NaN,NaN,NaN
2128,2023-06-22 04:49:36,NaN,NaN,NaN,NaN,NaN


In [131]:
data_1.activity.unique()

array(['click', 'lead', 'consultation', 'sale', nan], dtype=object)

In [132]:
random_values = ['click', 'lead', 'consultation', 'sale']

In [133]:
data_1['activity'] = random.choices(random_values, k=len(data_1))

In [134]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,johndoe@example.com,sale,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,janedoe@example.com,lead,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,janedoe@acme.com,consultation,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,alexsmith@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,NaN,click,NaN,NaN,NaN
2126,2023-06-28 10:26:58,NaN,consultation,NaN,NaN,NaN
2127,2023-06-19 22:47:34,NaN,sale,NaN,NaN,NaN
2128,2023-06-22 04:49:36,NaN,click,NaN,NaN,NaN


In [135]:
data_1['activity'].value_counts()

sale            570
consultation    538
click           521
lead            501
Name: activity, dtype: int64

In [136]:
data_1['customer'].value_counts(dropna=False)

NaN                       2000
janedoe@example.com         37
                            31
alexsmith@example.com       29
johndoe@example.com         23
janedoe@acme.com             8
sarahjones@example.com       1
sarahsmith@example.com       1
Name: customer, dtype: int64

In [137]:
customers =['john.doe@example.com',
'emma.smith@example.com',
'alex.wilson@example.com',
'laura.jones@example.com',
'michael.brown@example.com',
'sarah.davis@example.com',
'daniel.thomas@example.com',
'olivia.white@example.com',
'william.martin@example.com',
'emily.jackson@example.com']

data_1['customer'] = random.choices(customers, k=len(data_1))

In [138]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,alex.wilson@example.com,sale,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,laura.jones@example.com,lead,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,william.martin@example.com,consultation,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,emma.smith@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,john.doe@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,olivia.white@example.com,click,NaN,NaN,NaN
2126,2023-06-28 10:26:58,sarah.davis@example.com,consultation,NaN,NaN,NaN
2127,2023-06-19 22:47:34,sarah.davis@example.com,sale,NaN,NaN,NaN
2128,2023-06-22 04:49:36,emma.smith@example.com,click,NaN,NaN,NaN


In [139]:
landing = ['https://www.example.com/special-offer',
'https://www.example.com/limited-time-deal',
'https://www.example.com/exclusive-discount',
'https://www.example.com/new-product-launch',
'https://www.example.com/summer-sale']

def insert_random_landing_page(row):
    if row['activity'] == 'click':
        return random.choice(landing)
    else:
        return row['link']

# Use apply function to update the 'link' column
data_1['link'] = data_1.apply(insert_random_landing_page, axis=1)


In [140]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,alex.wilson@example.com,sale,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com
1,2023-06-01 09:10:00,laura.jones@example.com,lead,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,https://example.com/pricing
2,2023-06-01 09:15:00,william.martin@example.com,consultation,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,https://example.com/about-us
3,2023-06-01 09:20:00,emma.smith@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
4,2023-06-01 09:25:00,john.doe@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,https://example.com/products
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,olivia.white@example.com,click,NaN,NaN,https://www.example.com/special-offer
2126,2023-06-28 10:26:58,sarah.davis@example.com,consultation,NaN,NaN,NaN
2127,2023-06-19 22:47:34,sarah.davis@example.com,sale,NaN,NaN,NaN
2128,2023-06-22 04:49:36,emma.smith@example.com,click,NaN,NaN,https://www.example.com/summer-sale


In [141]:
mask = data_1['activity'] != 'click'

# Remove values from the 'link' column where activity is not equal to 'click'
data_1.loc[mask, 'link'] = ''

In [142]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,alex.wilson@example.com,sale,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,
1,2023-06-01 09:10:00,laura.jones@example.com,lead,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,
2,2023-06-01 09:15:00,william.martin@example.com,consultation,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,
3,2023-06-01 09:20:00,emma.smith@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,
4,2023-06-01 09:25:00,john.doe@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",NaN,
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,olivia.white@example.com,click,NaN,NaN,https://www.example.com/special-offer
2126,2023-06-28 10:26:58,sarah.davis@example.com,consultation,NaN,NaN,
2127,2023-06-19 22:47:34,sarah.davis@example.com,sale,NaN,NaN,
2128,2023-06-22 04:49:36,emma.smith@example.com,click,NaN,NaN,https://www.example.com/summer-sale


In [143]:
revenue = [9.99, 15.99, 29.99, 49.99, 99.99]

def insert_random_revenue(row):
    if row['activity'] == 'sale':
        return random.choice(revenue)
    else:
        return row['revenue_impact']

# Use apply function to update the 'link' column
data_1['revenue_impact'] = data_1.apply(insert_random_revenue, axis=1)

In [144]:
data_1

,ts,customer,activity,feature_json,revenue_impact,link
0,2023-06-01 09:00:00,alex.wilson@example.com,sale,"{""page"": ""homepage"", ""source"": ""organic""}",29.99,
1,2023-06-01 09:10:00,laura.jones@example.com,lead,"{""page"": ""pricing"", ""source"": ""referral""}",NaN,
2,2023-06-01 09:15:00,william.martin@example.com,consultation,"{""page"": ""homepage"", ""source"": ""organic""}",NaN,
3,2023-06-01 09:20:00,emma.smith@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",99.99,
4,2023-06-01 09:25:00,john.doe@example.com,sale,"{""page"": ""product_category"", ""source"": ""paid_ad""}",9.99,
...,...,...,...,...,...,...
2125,2023-06-08 15:42:57,olivia.white@example.com,click,NaN,NaN,https://www.example.com/special-offer
2126,2023-06-28 10:26:58,sarah.davis@example.com,consultation,NaN,NaN,
2127,2023-06-19 22:47:34,sarah.davis@example.com,sale,NaN,29.99,
2128,2023-06-22 04:49:36,emma.smith@example.com,click,NaN,NaN,https://www.example.com/summer-sale


In [145]:
data_1.to_csv('C:/Mike/Analytics/activity_schema_table/table_2k.csv', index=False)